USED PACKAGES

In [ ]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('courses.csv')
print(df)

      Course_ID   Course_Name      Category  Duration (hours)  \
0             1      Course_1  Office Tools                21   
1             2      Course_2  Office Tools                57   
2             3      Course_3    Technology                52   
3             4      Course_4  Office Tools                69   
4             5      Course_5    Technology                43   
...         ...           ...           ...               ...   
9995       9996   Course_9996            AI                87   
9996       9997   Course_9997  Data Science                96   
9997       9998   Course_9998            AI                25   
9998       9999   Course_9999   Programming                46   
9999      10000  Course_10000   Programming                50   

      Enrolled_Students  Completion_Rate (%)           Platform   Price ($)  \
0                  4217            50.646827           Coursera   38.797425   
1                  4238            82.240240                e